In [1]:
import os
import numpy as np
import tensorflow as tf
import skimage as sk
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import pickle


import models

PATH = '../data/landmarks/'

IMAGE_SIZE = (100,100,3)

# Limite size of input for testing
TEST_LIMIT = 1000

SPLIT = 0.8
BATCH_SIZE = 32
EPOCHS = 20
STEPS_PER_EPOCH = 40

## Dataset pre-processing

In [7]:
with open(PATH + 'dict.pickle', 'rb') as handle:
    dictionary = pickle.load(handle)

#     dictionary={
#     'filenames': string array with the full path,
#     'classes': array (len,1),
#     'bboxes': array (len,4),
#     'landmarks': array (len,10)
#     }
    
h,w,c=IMAGE_SIZE
input_images = np.empty((0,h,w,c))
for i in tqdm_notebook(range(len(dictionary['filenames']))):
#for i in tqdm_notebook(range(10)):
    image = sk.io.imread(dictionary['filenames'][i])
    input_images = np.vstack((input_images,[image]))

HBox(children=(IntProgress(value=0, max=2621), HTML(value='')))

In [8]:
alpha_bbox=0.5
alpha_landmarks=0.5

input_beta = dictionary['classes']

beta = np.expand_dims(input_beta, axis=1)

output_classes = dictionary['classes']
output_bboxes = alpha_bbox * np.hstack([beta]*4) * dictionary['bboxes']
output_landmarks = alpha_bbox * np.hstack([beta]*10) * dictionary['landmarks']

## Model definition

In [11]:
model = models.PNet()
print(model.summary())
losses = {
    'output_class': 'binary_crossentropy',
    'output_bbox': 'mse',
    'output_landmarks': 'mse'
}

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss=losses,
              metrics=['accuracy']) 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, None, None, 1 280         lambda_6[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, None, None, 1 0           conv2d_12[0][0]                  
__________________________________________________________________________________________________
conv2d_13 

## Training

In [5]:
%load_ext autoreload
%autoreload 1
%aimport models

In [ ]:
# model.fit(
#     images_train,
#     {"mask_output": masks_train, "landmarks_output": labels_train},
#     epochs=EPOCHS,
#     batch_size=BATCH_SIZE,
#     validation_data=(images_valid, {"mask_output": masks_valid, "landmarks_output": labels_valid})
# )

callbacks = [
#     tf.keras.callbacks.ModelCheckpoint(
#         '../weights/landmarks/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
#         save_best_only=True,
#         save_weights_only=True
#         ),
    tf.keras.callbacks.TensorBoard(log_dir='../output/logs')
    ]

model.fit(
    {'input_image':input_images, 'input_beta':input_beta},
    {'output_class':output_classes, 'output_bbox':output_bboxes, 'output_landmarks':output_landmarks},
    epochs=1000,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    callbacks=callbacks
)

Train on 2096 samples, validate on 525 samples
Epoch 1/1000


1312/2096 [=================>............] - ETA: 47s - loss: 70.2099 - output_class_loss: 0.7562 - output_bbox_loss: 0.0498 - output_landmarks_loss: 69.4040 - output_class_acc: 0.1562 - output_bbox_acc: 0.8438 - output_landmarks_acc: 0.843 - ETA: 27s - loss: 124.5106 - output_class_loss: 0.7342 - output_bbox_loss: 0.0669 - output_landmarks_loss: 123.7095 - output_class_acc: 0.2031 - output_bbox_acc: 0.8594 - output_landmarks_acc: 0.796 - ETA: 19s - loss: 133.4095 - output_class_loss: 0.7226 - output_bbox_loss: 0.0622 - output_landmarks_loss: 132.6247 - output_class_acc: 0.2188 - output_bbox_acc: 0.8750 - output_landmarks_acc: 0.781 - ETA: 16s - loss: 142.5027 - output_class_loss: 0.7159 - output_bbox_loss: 0.0556 - output_landmarks_loss: 141.7311 - output_class_acc: 0.2500 - output_bbox_acc: 0.8750 - output_landmarks_acc: 0.757 - ETA: 13s - loss: 150.4488 - output_class_loss: 0.7121 - output_bbox_loss: 0.0511 - output_landmarks_loss: 149.6856 - output_class_acc: 0.2625 - output_bbox_a

2096/2096 [==============================] - ETA: 2s - loss: 94.9000 - output_class_loss: 0.6907 - output_bbox_loss: 0.0841 - output_landmarks_loss: 94.1252 - output_class_acc: 0.6808 - output_bbox_acc: 0.8832 - output_landmarks_acc: 0.75 - ETA: 2s - loss: 93.9256 - output_class_loss: 0.6906 - output_bbox_loss: 0.0841 - output_landmarks_loss: 93.1508 - output_class_acc: 0.6817 - output_bbox_acc: 0.8808 - output_landmarks_acc: 0.74 - ETA: 2s - loss: 92.1486 - output_class_loss: 0.6903 - output_bbox_loss: 0.0830 - output_landmarks_loss: 91.3754 - output_class_acc: 0.6875 - output_bbox_acc: 0.8828 - output_landmarks_acc: 0.75 - ETA: 2s - loss: 90.5902 - output_class_loss: 0.6900 - output_bbox_loss: 0.0818 - output_landmarks_loss: 89.8185 - output_class_acc: 0.6917 - output_bbox_acc: 0.8840 - output_landmarks_acc: 0.75 - ETA: 2s - loss: 90.1124 - output_class_loss: 0.6898 - output_bbox_loss: 0.0825 - output_landmarks_loss: 89.3401 - output_class_acc: 0.6923 - output_bbox_acc: 0.8818 - outp

1344/2096 [==================>...........] - ETA: 5s - loss: 25.2554 - output_class_loss: 0.6742 - output_bbox_loss: 0.0446 - output_landmarks_loss: 24.5366 - output_class_acc: 0.8125 - output_bbox_acc: 0.8438 - output_landmarks_acc: 0.81 - ETA: 5s - loss: 27.2460 - output_class_loss: 0.6761 - output_bbox_loss: 0.0605 - output_landmarks_loss: 26.5095 - output_class_acc: 0.7812 - output_bbox_acc: 0.8281 - output_landmarks_acc: 0.78 - ETA: 5s - loss: 31.6945 - output_class_loss: 0.6773 - output_bbox_loss: 0.0844 - output_landmarks_loss: 30.9328 - output_class_acc: 0.7604 - output_bbox_acc: 0.8021 - output_landmarks_acc: 0.76 - ETA: 5s - loss: 35.9880 - output_class_loss: 0.6763 - output_bbox_loss: 0.0809 - output_landmarks_loss: 35.2308 - output_class_acc: 0.7734 - output_bbox_acc: 0.8203 - output_landmarks_acc: 0.77 - ETA: 5s - loss: 32.9372 - output_class_loss: 0.6752 - output_bbox_loss: 0.0713 - output_landmarks_loss: 32.1907 - output_class_acc: 0.7875 - output_bbox_acc: 0.8375 - outp

2096/2096 [==============================] - ETA: 2s - loss: 28.3838 - output_class_loss: 0.6731 - output_bbox_loss: 0.0828 - output_landmarks_loss: 27.6279 - output_class_acc: 0.7558 - output_bbox_acc: 0.8270 - output_landmarks_acc: 0.75 - ETA: 2s - loss: 28.0349 - output_class_loss: 0.6728 - output_bbox_loss: 0.0824 - output_landmarks_loss: 27.2797 - output_class_acc: 0.7578 - output_bbox_acc: 0.8281 - output_landmarks_acc: 0.75 - ETA: 1s - loss: 27.8911 - output_class_loss: 0.6726 - output_bbox_loss: 0.0821 - output_landmarks_loss: 27.1364 - output_class_acc: 0.7583 - output_bbox_acc: 0.8278 - output_landmarks_acc: 0.75 - ETA: 1s - loss: 27.5466 - output_class_loss: 0.6723 - output_bbox_loss: 0.0811 - output_landmarks_loss: 26.7932 - output_class_acc: 0.7602 - output_bbox_acc: 0.8302 - output_landmarks_acc: 0.76 - ETA: 1s - loss: 27.5649 - output_class_loss: 0.6723 - output_bbox_loss: 0.0818 - output_landmarks_loss: 26.8107 - output_class_acc: 0.7586 - output_bbox_acc: 0.8305 - outp

...

1344/2096 [==================>...........] - ETA: 6s - loss: 19.8207 - output_class_loss: 0.7338 - output_bbox_loss: 0.0348 - output_landmarks_loss: 19.0521 - output_class_acc: 0.5938 - output_bbox_acc: 0.8125 - output_landmarks_acc: 0.84 - ETA: 5s - loss: 19.8180 - output_class_loss: 0.6995 - output_bbox_loss: 0.0296 - output_landmarks_loss: 19.0889 - output_class_acc: 0.6250 - output_bbox_acc: 0.8125 - output_landmarks_acc: 0.79 - ETA: 5s - loss: 15.9260 - output_class_loss: 0.6195 - output_bbox_loss: 0.0209 - output_landmarks_loss: 15.2856 - output_class_acc: 0.6979 - output_bbox_acc: 0.8646 - output_landmarks_acc: 0.85 - ETA: 5s - loss: 15.6990 - output_class_loss: 0.6224 - output_bbox_loss: 0.0177 - output_landmarks_loss: 15.0589 - output_class_acc: 0.6953 - output_bbox_acc: 0.8906 - output_landmarks_acc: 0.85 - ETA: 5s - loss: 14.1291 - output_class_loss: 0.6035 - output_bbox_loss: 0.0154 - output_landmarks_loss: 13.5101 - output_class_acc: 0.7125 - output_bbox_acc: 0.9062 - outp

2096/2096 [==============================] - ETA: 2s - loss: 11.2948 - output_class_loss: 0.5559 - output_bbox_loss: 0.0114 - output_landmarks_loss: 10.7275 - output_class_acc: 0.7558 - output_bbox_acc: 0.9288 - output_landmarks_acc: 0.89 - ETA: 2s - loss: 11.1951 - output_class_loss: 0.5553 - output_bbox_loss: 0.0113 - output_landmarks_loss: 10.6284 - output_class_acc: 0.7564 - output_bbox_acc: 0.9297 - output_landmarks_acc: 0.89 - ETA: 1s - loss: 11.0854 - output_class_loss: 0.5547 - output_bbox_loss: 0.0112 - output_landmarks_loss: 10.5196 - output_class_acc: 0.7569 - output_bbox_acc: 0.9299 - output_landmarks_acc: 0.89 - ETA: 1s - loss: 10.9805 - output_class_loss: 0.5519 - output_bbox_loss: 0.0110 - output_landmarks_loss: 10.4176 - output_class_acc: 0.7595 - output_bbox_acc: 0.9300 - output_landmarks_acc: 0.89 - ETA: 1s - loss: 10.9744 - output_class_loss: 0.5499 - output_bbox_loss: 0.0110 - output_landmarks_loss: 10.4135 - output_class_acc: 0.7613 - output_bbox_acc: 0.9309 - outp

1344/2096 [==================>...........] - ETA: 5s - loss: 4.0154 - output_class_loss: 0.4591 - output_bbox_loss: 0.0037 - output_landmarks_loss: 3.5526 - output_class_acc: 0.8438 - output_bbox_acc: 1.0000 - output_landmarks_acc: 0.90 - ETA: 5s - loss: 8.0329 - output_class_loss: 0.5279 - output_bbox_loss: 0.0057 - output_landmarks_loss: 7.4992 - output_class_acc: 0.7812 - output_bbox_acc: 0.9844 - output_landmarks_acc: 0.93 - ETA: 5s - loss: 6.4937 - output_class_loss: 0.4935 - output_bbox_loss: 0.0072 - output_landmarks_loss: 5.9930 - output_class_acc: 0.8125 - output_bbox_acc: 0.9688 - output_landmarks_acc: 0.92 - ETA: 5s - loss: 8.5516 - output_class_loss: 0.5279 - output_bbox_loss: 0.0087 - output_landmarks_loss: 8.0150 - output_class_acc: 0.7812 - output_bbox_acc: 0.9531 - output_landmarks_acc: 0.92 - ETA: 5s - loss: 7.9809 - output_class_loss: 0.5417 - output_bbox_loss: 0.0082 - output_landmarks_loss: 7.4310 - output_class_acc: 0.7688 - output_bbox_acc: 0.9437 - output_landmar

2096/2096 [==============================] - ETA: 2s - loss: 10.7299 - output_class_loss: 0.5631 - output_bbox_loss: 0.0119 - output_landmarks_loss: 10.1548 - output_class_acc: 0.7493 - output_bbox_acc: 0.9302 - output_landmarks_acc: 0.89 - ETA: 1s - loss: 10.7167 - output_class_loss: 0.5639 - output_bbox_loss: 0.0119 - output_landmarks_loss: 10.1409 - output_class_acc: 0.7486 - output_bbox_acc: 0.9290 - output_landmarks_acc: 0.89 - ETA: 1s - loss: 10.7543 - output_class_loss: 0.5654 - output_bbox_loss: 0.0119 - output_landmarks_loss: 10.1769 - output_class_acc: 0.7472 - output_bbox_acc: 0.9292 - output_landmarks_acc: 0.89 - ETA: 1s - loss: 10.9234 - output_class_loss: 0.5653 - output_bbox_loss: 0.0122 - output_landmarks_loss: 10.3459 - output_class_acc: 0.7473 - output_bbox_acc: 0.9293 - output_landmarks_acc: 0.89 - ETA: 1s - loss: 11.1188 - output_class_loss: 0.5697 - output_bbox_loss: 0.0122 - output_landmarks_loss: 10.5370 - output_class_acc: 0.7434 - output_bbox_acc: 0.9275 - outp

 704/2096 [=========>....................] - ETA: 5s - loss: 9.3541 - output_class_loss: 0.5281 - output_bbox_loss: 0.0074 - output_landmarks_loss: 8.8187 - output_class_acc: 0.7812 - output_bbox_acc: 1.0000 - output_landmarks_acc: 0.90 - ETA: 5s - loss: 9.5594 - output_class_loss: 0.5795 - output_bbox_loss: 0.0083 - output_landmarks_loss: 8.9717 - output_class_acc: 0.7344 - output_bbox_acc: 0.9375 - output_landmarks_acc: 0.87 - ETA: 5s - loss: 10.9869 - output_class_loss: 0.5623 - output_bbox_loss: 0.0081 - output_landmarks_loss: 10.4164 - output_class_acc: 0.7500 - output_bbox_acc: 0.9479 - output_landmarks_acc: 0.89 - ETA: 5s - loss: 14.4120 - output_class_loss: 0.5966 - output_bbox_loss: 0.0176 - output_landmarks_loss: 13.7978 - output_class_acc: 0.7188 - output_bbox_acc: 0.9219 - output_landmarks_acc: 0.88 - ETA: 5s - loss: 14.9415 - output_class_loss: 0.6172 - output_bbox_loss: 0.0190 - output_landmarks_loss: 14.3053 - output_class_acc: 0.7000 - output_bbox_acc: 0.9125 - output_l